In [33]:
import pandas as pd
import requests
import json
import os
import datetime
import numpy as np

In [4]:
os.makedirs("./idw", exist_ok=True)

In [6]:
r_estaciones = requests.api.get("https://www.piraguacorantioquia.com.co/api/v1/estaciones")

In [9]:
estaciones = json.loads(r_estaciones.content)

In [18]:
df_ppt = pd.DataFrame({int(est["codigo"]):{"lat":float(est["latitud"]), "lon":float(est["longitud"])} for est in estaciones["estaciones"] if str(est["tipo"]) == "1"}).T

In [20]:
now = datetime.datetime.now()

In [26]:
start_date = (now - pd.Timedelta("24h")).strftime("%Y-%m-%d")


In [63]:
for i in df_ppt.index:
    r_ppt = requests.api.get(f"https://www.piraguacorantioquia.com.co/api/v1/estaciones/{i}/precipitacion?fecha__gte={start_date}&page=2&size=288")

    data = json.loads(r_ppt.content)
    
    try:
        df_ppt.loc[i, "ppt"] = np.array([i["muestra"] for i in data["results"] if float(i["muestra"]) >= 0]).astype(float).sum()
    except:
        print(i)

26
34
44
48
5
68
73
74
8
81
84
87
88
93
94


In [101]:
df_ppt.dropna().to_csv(f"{start_date}.csv", index = False)

In [102]:
header = f"""<OGRVRTDataSource>
    <OGRVRTLayer name="dem">
        <SrcDataSource>{start_date}.csv</SrcDataSource>
        <GeometryType>wkbPoint</GeometryType>
        <GeometryField encoding="PointFromColumns" x="lon" y="lat" z="ppt"/>
    </OGRVRTLayer>
</OGRVRTDataSource>"""

In [103]:
with open(f"{start_date}.vrt", "w") as f:
    f.write(header)

In [104]:
os.system(f"gdal_grid -zfield 'ppt' -a invdist:power=2.0:smoothing=1.0  -outsize 400 400 -of GTiff -ot Float64 -l dem {start_date}.vrt {start_date}.tiff")


Failed to find field ppt on layer dem, skipping.


ERROR 1: Failed to find layer 'dem' on datasource '2022-05-22.csv'.


256

In [105]:
df_ppt

,lat,lon,ppt
1,7.492328,-74.868139,0.000
10,5.855444,-75.826167,0.000
11,6.059778,-75.792333,0.000
12,5.596583,-75.813861,0.000
13,5.655417,-75.882556,0.000
...,...,...,...
94,6.951863,-73.958170,NaN
95,5.911333,-75.659417,0.000
96,6.842278,-75.217667,0.254
97,7.063944,-75.685972,0.000
